In [ ]:
import skimage.io as io
import cv2
import numpy as np
from zipfile import ZipFile
import pandas as pd
import torch

In [ ]:
file_name='labels/coco2017labels.zip'
with ZipFile(file_name, 'r') as zip:
#     bbox=zip.read('coco/labels/train2017/000000371735.txt')
#     box=(bbox.decode("utf-8"))
#     box=pd.DataFrame([x.split() for x in box.rstrip('\n').split('\n')],columns=['class','xc','yc','w','h'])
#     print(box)
    count = len(zip.infolist())
    print()

In [ ]:
from zipfile import ZipFile
file_name='labels/coco2017labels.zip'
zip_file = ZipFile(file_name)

dfs = {text_file.filename: pd.read_csv(zip_file.open(text_file.filename))
       for text_file in zip_file.infolist()}

In [35]:
import torch

a=torch.tensor([[[1,2,2,4,0,0,0,1],[2,3,5,3,0,0,1,0]]])
print(a.shape)
b=((a[:,:,4:]==1).nonzero())
b=b[:,-1].unsqueeze(0).unsqueeze(-1)
print(b.shape)
new=a[:,:,:4]
print(new.shape)
new=torch.cat((new.T,b.T)).T
print(new)
sorted_pred=torch.sort(new[0,:,4],descending=False)
print(sorted_pred)

torch.Size([1, 2, 8])
torch.Size([1, 2, 1])
torch.Size([1, 2, 4])
tensor([[[1, 2, 2, 4, 3],
         [2, 3, 5, 3, 2]]])
torch.return_types.sort(
values=tensor([2, 3]),
indices=tensor([1, 0]))


In [ ]:
b = box.values.astype(np.float32)
b=torch.tensor(b)
labels = b.T[0].reshape(b.shape[0], 1)
one_hot_target = (labels == torch.arange(80).reshape(1, 80)).float()
conf=torch.ones([b.shape[0],1])
boxes=torch.cat((b.T[1:],conf.T,one_hot_target.T)).T




In [ ]:
file_name='images/train2017.zip'
with ZipFile(file_name, 'r') as zip:
    image=zip.read('train2017/000000408542.jpg')
    img = cv2.imdecode(np.frombuffer(image, np.uint8),1)
    im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    io.imshow(im_rgb)
    io.imsave('fig.png',im_rgb)

In [ ]:
df=pd.read_csv('pointers/train2017.txt',names=['filename'])

In [ ]:
df['box']=df['filename'].apply(lambda x: 'coco/labels/'+x.split('.')[0]+'.txt')

In [ ]:
df['box']

In [1]:
from dataset import *
import timeit 
import skimage.io as io
import cv2
import numpy as np
from zipfile import ZipFile
import pandas as pd
import torch
import time
from darknet import *
import darknet as dn
import util as util
import torch.optim as optim
import sys
import timeit
import torch.autograd
import helper as helper
from torch.utils.tensorboard import SummaryWriter



net = Darknet("../cfg/yolov3.cfg")
inp_dim=net.inp_dim
pw_ph=net.pw_ph.to(device='cuda')
cx_cy=net.cx_cy.to(device='cuda')
stride=net.stride.to(device='cuda')


'''
when loading weights from dataparallel model then, you first need to instatiate the dataparallel model 
if you start fresh then first model.load_weights and then make it parallel
'''
try:
    PATH = '../mse.pth'
    weights = torch.load(PATH)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Assuming that we https://pytorch.org/docs/stable/data.html#torch.utils.data.Datasetare on a CUDA machine, this should print a CUDA device:
    print(device)
    net.to(device)

    if torch.cuda.device_count() > 9:
        print("Using ", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(net)
        model.to(device)
        model.load_state_dict(weights)
    else:
        model=net
        model.to(device)
        model.load_state_dict(weights)
        
except FileNotFoundError: 
#     net.load_weights("yolov3.weights")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Assuming that we are on a CUDA machine, this should print a CUDA device:

    print(device)
    net.to(device)
    if torch.cuda.device_count() > 1:
        print("Using ", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(net)
        model.to(device)
    else:
        model=net
        
        
transformed_dataset=Coco(partition='train',
                                           transform=transforms.Compose([
                                            ResizeToTensor(inp_dim)
                                           ]))


writer = SummaryWriter('../results/avg_cur_msemean')
dataset_len=(len(transformed_dataset))
print('Length of dataset is '+ str(dataset_len)+'\n')
batch_size=8

dataloader = DataLoader(transformed_dataset, batch_size=batch_size,
                        shuffle=True,collate_fn=helper.my_collate, num_workers=1)


optimizer = optim.SGD(model.parameters(), lr=0.000001, weight_decay=0.005, momentum=0.9)
epochs=50
total_loss=0
write=0
misses=0
break_flag=0
avg_iou=0
for e in range(epochs):
    prg_counter=0
    train_counter=0
    total_loss=0
    avg_iou=0
    avg_infs=0
    avg_conf=0
    avg_no_conf=0
    avg_pos=0
    avg_neg=0
    print("\n epoch "+str(e))
    misses=0
    for images,targets in dataloader:
        optimizer.zero_grad()
        targets,anchors,offset,strd,mask=helper.collapse_boxes(targets,pw_ph,cx_cy,stride)
        images=images.cuda()
        raw_pred = model(images, torch.cuda.is_available())
        raw_pred=helper.expand_predictions(raw_pred,mask)
        true_pred=util.transform(raw_pred.clone(),anchors,offset,strd)
        targets=targets.unsqueeze(-3)
        targets=targets.cuda()
        iou_mask,noobj_mask=util.get_responsible_masks(true_pred,targets,offset,strd,mask,inp_dim)
        
        iou=torch.diag(util.bbox_iou(util.get_abs_coord(true_pred[iou_mask.T,:].unsqueeze(-3)),util.transpose_target(get_abs_coord(targets)*inp_dim))).mean().item()
        
        
        noobj_box=raw_pred[:,:,4:5].clone()
        noobj_box=helper.uncollapse(noobj_box,mask)
        noobj_mask=helper.uncollapse(noobj_mask.T.unsqueeze(-1),mask)
        
        
        conf=raw_pred[iou_mask.T,:][:,4:5].mean().item()
        class_mask=targets[:,:,5:].type(torch.BoolTensor).squeeze(0)
        if(iou_mask.sum()==class_mask.shape[0]):
            pos_class=raw_pred[iou_mask.T,:][:,5:][class_mask].mean().item()
            neg_class=raw_pred[iou_mask.T,:][:,5:][~class_mask].mean().item()
        else:
            pos_class=0
            neg_class=0
        noobj_box=noobj_box[noobj_mask]
        no_obj_conf=noobj_box.mean().item()
        
        raw_pred=raw_pred[iou_mask.T,:]
        anchors=anchors[iou_mask.T,:]
        offset=offset[iou_mask.T,:]
        strd=strd[iou_mask.T,:]
        if(strd.shape[0]==sum(mask)):#this means that iou_mask failed and was all true, because max of zeros is true for all lenght of mask strd
            targets[:,:,0:4]=targets[:,:,0:4]*inp_dim
            targets=targets.squeeze(0)
            targets[:,0:4]=util.transform_groundtruth(targets,anchors,offset,strd)
            loss=util.yolo_loss(raw_pred,targets,noobj_box,batch_size)
            loss.backward()
            optimizer.step()
            
            avg_conf=avg_conf+conf
            avg_no_conf=avg_no_conf+no_obj_conf
            avg_pos=avg_pos+pos_class
            avg_neg=avg_neg+neg_class
            total_loss=total_loss+loss.item()
            avg_iou=avg_iou+iou
            sys.stdout.write('\rPgr:'+str(prg_counter/dataset_len*100*batch_size)+'%' ' L:'+ str(loss.item()))
            sys.stdout.write(' IoU:' +str(iou)+' pob:'+str(conf)+ ' nob:'+str(no_obj_conf))
            sys.stdout.write(' PCls:' +str(pos_class)+' ncls:'+str(neg_class))
            sys.stdout.flush()
            prg_counter=prg_counter+1
            train_counter=train_counter+1
        else:
            print('missed')
            print(strd.shape[0])
            prg_counter=prg_counter+1
        writer.add_scalar('AvLoss/train', total_loss/train_counter, train_counter)

        
        writer.add_scalar('AvIoU/train', avg_iou/train_counter, train_counter)

        writer.add_scalar('AvPConf/train', avg_conf/train_counter, train_counter)
        
        writer.add_scalar('AvNConf/train', avg_no_conf/train_counter, train_counter)
        
        writer.add_scalar('AvClass/train', avg_pos/train_counter, train_counter)
        
        writer.add_scalar('AvNClass/train', avg_neg/train_counter, train_counter)
        
        del loss, raw_pred, targets, true_pred, images,iou,noobj_box,conf
        torch.cuda.empty_cache()
    if misses>0:
        break
    torch.save(model.state_dict(), PATH)
#     writer.add_scalar('Loss/train', total_loss/train_counter, e)
#     writer.add_scalar('AIoU/train', avg_iou/train_counter, e)
#     writer.add_scalar('PConf/train', avg_conf/train_counter, e)
#     writer.add_scalar('NConf/train', avg_no_conf/train_counter, e)
#     writer.add_scalar('PClass/train', avg_pos/train_counter, e)
#     writer.add_scalar('NClass/train', avg_neg/train_counter, e)
    print('\ntotal number of misses is ' + str(misses))
    print('\n total average loss is '+str(total_loss/train_counter))
    print('\n total average iou is '+str(avg_iou/train_counter))

cuda:0
Length of dataset is 118287


 epoch 0
(427, 640, 3)
(337, 504, 3)
(428, 640, 3)
(480, 640, 3)
(640, 640, 3)
(640, 480, 3)
(640, 640, 3)
(480, 640, 3)
(500, 375, 3)
(480, 640, 3)
(427, 640, 3)
(477, 640, 3)
(428, 640, 3)
(427, 640, 3)
(480, 640, 3)
(427, 640, 3)
(640, 480, 3)
(480, 640, 3)
(407, 640, 3)
(427, 640, 3)
(426, 640, 3)
(427, 640, 3)
(640, 427, 3)
(427, 640, 3)


/users/konsa15/.conda/envs/faster_rcnn/lib/python3.8/site-packages/torch/nn/functional.py:2503: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn("Default upsampling behavior when mode={} is changed "


Pgr:0.0% L:133.01141357421875 IoU:0.8107700943946838 pob:0.44330593943595886 nob:0.0008358145714737475 PCls:0.6715307235717773 ncls:0.003973086830228567(424, 640, 3)
(500, 375, 3)
(640, 427, 3)
(480, 640, 3)
(426, 640, 3)
(483, 640, 3)
(429, 300, 3)
(480, 640, 3)
Pgr:0.006763211510985992% L:236.9548797607422 IoU:0.7679412364959717 pob:0.3353475332260132 nob:0.0011811520671471953 PCls:0.6919551491737366 ncls:0.0036552429664880037(479, 640, 3)
(640, 640, 3)
(333, 500, 3)
Pgr:0.013526423021971984% L:352.5690612792969 IoU:0.7271129488945007 pob:0.21772557497024536 nob:0.000831308017950505 PCls:0.6183369755744934 ncls:0.004896657075732946(427, 640, 3)
(480, 640, 3)
(333, 500, 3)
(427, 640, 3)
(425, 640, 3)
(480, 640, 3)
(427, 640, 3)
(480, 640, 3)
Pgr:0.020289634532957977% L:328.4309387207031 IoU:0.7310498952865601 pob:0.2989698648452759 nob:0.001124958973377943 PCls:0.7640601992607117 ncls:0.0033408617600798607(375, 500, 3)
(388, 640, 3)
(640, 640, 3)
(640, 425, 3)
(640, 640, 3)
(480, 640,

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/users/konsa15/.conda/envs/faster_rcnn/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/users/konsa15/.conda/envs/faster_rcnn/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/users/konsa15/.conda/envs/faster_rcnn/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/mnt/data1/users/konsa15/workspace/notebooks/coco/yolo/dataset.py", line 40, in __getitem__
    with open(label_path) as data:
FileNotFoundError: [Errno 2] No such file or directory: '../labels/coco/labels/train2017/000000434129.txt'


In [7]:

avg_neg/train_counter

0.003559733550560331

In [12]:
targets2,anchors,offset,strd,mask=helper.collapse_boxes(targets,pw_ph,cx_cy,stride)
transformed_output=true_pred
targets=targets
offset=offset
strd=strd
mask=mask
inp_dim
'''
this function takes the transformed_output and
the target box in respect to the resized image size
and returns a mask which can be applied to select the 
best raw input,anchors and cx_cy_offset
and the noobj_mask for the negatives
targets is a list
'''
#first transpose the centered normalised target coords
centered_target=transpose_target(targets)[:,:,0:2]
#multiply by inp_dim then devide by stride to get the relative grid size coordinates, floor the result to get the corresponding cell
centered_target=torch.floor(centered_target*inp_dim/strd)
#create a mask to find where the gt falls into which gridcell in the grid coordinate system
fall_into_mask=centered_target==offset
fall_into_mask=fall_into_mask[:,:,0]&fall_into_mask[:,:,1]
#     fall_into_mask= ~fall_into_mask
#create a copy of the transformed output
best_bboxes=transformed_output.clone()
#apply reverse mask to copy in order to zero all other bbox locations
best_bboxes[~fall_into_mask]=0   
#transform the copy to xmin,xmax,ymin,ymax
best_responsible_coord=get_abs_coord(best_bboxes)
targets=transpose_target(get_abs_coord(targets))*inp_dim
#calculate best iou and mask
responsible_iou=bbox_iou(best_responsible_coord,targets,True)

responsible_iou[responsible_iou.ne(responsible_iou)] = 0
responsible_mask=responsible_iou.max(dim=0)[0] == responsible_iou

print(responsible_mask.shape)

abs_coord=get_abs_coord(transformed_output)
iou=bbox_iou(abs_coord,targets,True)
iou[iou.ne(iou)] = 0
ignore_mask=0.5<=iou
inverted_mask=iou.max(dim=0)[0] != iou
noobj_mask=~same_picture_mask(responsible_mask.clone()|ignore_mask,mask)

torch.Size([10647, 78])


In [11]:
print(responsible_mask)
if(responsible_mask.sum()>sum(mask)):
    print('jello')
    responsible_mask1=correct_iou_mask(responsible_mask,fall_into_mask)
print(responsible_mask1.shape)

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]], device='cuda:0')
torch.Size([10647, 78])


In [16]:
a=torch.randint(0, 9, (1,)) == torch.arange(9)
a=a.repeat(5)
a=a.repeat(2,1)
print(a)
print(a.T)

tensor([[False,  True, False, False, False, False, False, False, False, False,
          True, False, False, False, False, False, False, False, False,  True,
         False, False, False, False, False, False, False, False,  True, False,
         False, False, False, False, False, False, False,  True, False, False,
         False, False, False, False, False],
        [False,  True, False, False, False, False, False, False, False, False,
          True, False, False, False, False, False, False, False, False,  True,
         False, False, False, False, False, False, False, False,  True, False,
         False, False, False, False, False, False, False,  True, False, False,
         False, False, False, False, False]])
tensor([[False, False],
        [ True,  True],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [ True,  True],
        [Fals

tensor([False, False, False, False, False, False, False,  True, False])


In [68]:
le_mask=(responsible_mask.sum(axis=0)==responsible_mask.sum(axis=0).max())

In [81]:
responsible_mask[:,le_mask]=(responsible_mask[:,le_mask]&fall_into_mask[le_mask,:].T)

In [2]:
print(le_mask)
print(fall_into_mask[le_mask,:].shape)


NameError: name 'le_mask' is not defined

In [11]:
a=torch.tensor([1,2,3,4,5,6,6,6])
indices=(((a==a.max())==True).nonzero())
for ind in indices:
    print(ind)

tensor([5])
tensor([6])
tensor([7])


In [1]:
from darknet import *
import darknet as dn
import util as util
import torch.optim as optim
import pandas as pd
import time
import sys
import timeit
from dataset import *
import torchvision.ops.boxes as nms_box
import helper as helper
from utils import *
from BoundingBox import BoundingBox
from BoundingBoxes import BoundingBoxes


net = Darknet("../cfg/yolov3.cfg")
inp_dim=net.inp_dim
pw_ph=net.pw_ph
cx_cy=net.cx_cy
stride=net.stride


'''
when loading weights from dataparallel model then, you first need to instatiate the dataparallel model 
if you start fresh then first model.load_weights and then make it parallel
'''
try:
    PATH = '../mse.pth'
    weights = torch.load(PATH)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Assuming that we are on a CUDA machine, this should print a CUDA device:
    print(device)
    net.to(device)

    if torch.cuda.device_count() > 9:
        print("Using ", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(net)
    else:
        model=net

    model.to(device)
    
    
    model.load_state_dict(weights)
    model.eval()
except FileNotFoundError: 
    net.load_weights("../yolov3.weights")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Assuming that we are on a CUDA machine, this should print a CUDA device:

    print(device)
    net.to(device)
    if torch.cuda.device_count() > 1:
      print("Using ", torch.cuda.device_count(), "GPUs!")
      model = nn.DataParallel(net)

    model.to(device)
    




print('testing with '+ PATH +'\n')
transformed_dataset=Coco(partition='val',
                                           transform=transforms.Compose([
                                            ResizeToTensor(inp_dim)
                                           ]))



dataset_len=(len(transformed_dataset))
print('Length of dataset is '+ str(dataset_len)+'\n')
batch_size=8

dataloader = DataLoader(transformed_dataset, batch_size=batch_size,
                        shuffle=True,collate_fn=helper.my_collate, num_workers=2)

true_pos=0
false_pos=0
counter=0
iou_threshold=0
confidence=0.25
recall_counter=0

for images,targets,img_name in dataloader:
    inp=images.cuda()
    raw_pred = model(inp, torch.cuda.is_available())
    targets,anchors,offset,strd,mask=helper.collapse_boxes(targets,pw_ph,cx_cy,stride)
    
    raw_pred=raw_pred.to(device='cuda')
    true_pred=util.transform(raw_pred.clone(),pw_ph,cx_cy,stride)
    
    sorted_pred=torch.sort(true_pred[:,:,4],descending=True)
    pred_mask=sorted_pred[0]>confidence
    indices=[(sorted_pred[1][e,:][pred_mask[e,:]]) for e in range(pred_mask.shape[0])]
    pred_final=[true_pred[i,indices[i],:] for i in range(len(indices))]
    
    pred_final_coord=[util.get_abs_coord(pred_final[i].unsqueeze(-2)) for i in range(len(pred_final))]
    
    indices=[nms_box.nms(pred_final_coord[i][0],pred_final[i][:,4],iou_threshold) for i in range(len(pred_final))]

    pred_final=[pred_final[i][indices[i],:] for i in range(len(pred_final))]
    
#     pred_final[:,0:4]=pred_final[:,0:4]/inp_dim
    helper.write_pred(img_name,pred_final,inp_dim)
    

    
# Read txt files containing bounding boxes (ground truth and detections)


cuda:0
testing with ../mse.pth

Length of dataset is 5000



/users/konsa15/.conda/envs/faster_rcnn/lib/python3.8/site-packages/torch/nn/functional.py:2503: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn("Default upsampling behavior when mode={} is changed "


In [2]:
from darknet import *
import darknet as dn
import util as util
import torch.optim as optim
import pandas as pd
import time
import sys
import timeit
from dataset import *
import torchvision.ops.boxes as nms_box
import helper as helper
from utils import *
from BoundingBox import BoundingBox
from BoundingBoxes import BoundingBoxes
boundingboxes = helper.getBoundingBoxes()


In [6]:
from Evaluator import *
from utils import *
from BoundingBox import BoundingBox
from BoundingBoxes import BoundingBoxes


boundingboxes = helper.getBoundingBoxes()

evaluator = Evaluator()


metricsPerClass = evaluator.GetPascalVOCMetrics(boundingboxes, IOUThreshold=0.5)
print("Average precision values per class:\n")
# Loop through classes to obtain their metrics
mAP=0
counter=0
for mc in metricsPerClass:
    # Get metric values per each class
    c = mc['class']
    precision = mc['precision']
    recall = mc['recall']
    average_precision = mc['AP']
    ipre = mc['interpolated precision']
    irec = mc['interpolated recall']
    # Print AP per class
    mAP=average_precision+mAP
    print('%s: %f' % (c, average_precision))

print('map is:',mAP/80)

Average precision values per class:

0: 0.359763
1: 0.076982
10: 0.243045
11: 0.397133
12: 0.055159
13: 0.043901
14: 0.099771
15: 0.254480
16: 0.125770
17: 0.167524
18: 0.110603
19: 0.165715
2: 0.220375
20: 0.279541
21: 0.138327
22: 0.482143
23: 0.456795
24: 0.003953
25: 0.050659
26: 0.000000
27: 0.039617
28: 0.013829
29: 0.086616
3: 0.154901
30: 0.008983
31: 0.000000
32: 0.211431
33: 0.265283
34: 0.003448
35: 0.024204
36: 0.012597
37: 0.016695
38: 0.020458
39: 0.052578
4: 0.417461
40: 0.042928
41: 0.069803
42: 0.008140
43: 0.009813
44: 0.000000
45: 0.075122
46: 0.044401
47: 0.004237
48: 0.024868
49: 0.050960
5: 0.321404
50: 0.040070
51: 0.013454
52: 0.007733
53: 0.159415
54: 0.072998
55: 0.018806
56: 0.047063
57: 0.148153
58: 0.040633
59: 0.131349
6: 0.403184
60: 0.049885
61: 0.343441
62: 0.278768
63: 0.177673
64: 0.132350
65: 0.005452
66: 0.158539
67: 0.020208
68: 0.246105
69: 0.065828
7: 0.081207
70: 0.000000
71: 0.118368
72: 0.138725
73: 0.022412
74: 0.334198
75: 0.044923
76: 0.000

[0,
 0.0015772870662460567,
 0.0031545741324921135,
 0.00473186119873817,
 0.006309148264984227,
 0.007886435331230283,
 0.00946372239747634,
 0.011041009463722398,
 0.012618296529968454,
 0.014195583596214511,
 0.015772870662460567,
 0.015772870662460567,
 0.017350157728706624,
 0.01892744479495268,
 0.02050473186119874,
 0.022082018927444796,
 0.02365930599369085,
 0.025236593059936908,
 0.026813880126182965,
 0.026813880126182965,
 0.028391167192429023,
 0.028391167192429023,
 0.02996845425867508,
 0.031545741324921134,
 0.033123028391167195,
 0.03470031545741325,
 0.03470031545741325,
 0.03470031545741325,
 0.03627760252365931,
 0.03785488958990536,
 0.03785488958990536,
 0.03943217665615142,
 0.03943217665615142,
 0.03943217665615142,
 0.04100946372239748,
 0.04258675078864353,
 0.04416403785488959,
 0.04574132492113565,
 0.0473186119873817,
 0.04889589905362776,
 0.050473186119873815,
 0.052050473186119876,
 0.052050473186119876,
 0.052050473186119876,
 0.052050473186119876,
 0.0

In [14]:
from Evaluator import *
evaluator = Evaluator()
metricsPerClass = evaluator.GetPascalVOCMetrics(boundingboxes, IOUThreshold=0.5)
print("Average precision values per class:\n")
# Loop through classes to obtain their metrics
mAP=0
counter=0
for mc in metricsPerClass:
    # Get metric values per each class
    c = mc['class']
    precision = mc['precision']
    recall = mc['recall']
    average_precision = mc['AP']
    ipre = mc['interpolated precision']
    irec = mc['interpolated recall']
    # Print AP per class
    mAP=average_precision+mAP
    print('%s: %f' % (c, average_precision))

print('map is:',mAP/80)

Average precision values per class:

0: 0.326917
1: 0.061502
10: 0.235790
11: 0.387210
12: 0.046825
13: 0.037967
14: 0.090984
15: 0.248136
16: 0.117549
17: 0.156873
18: 0.087636
19: 0.149208
2: 0.190256
20: 0.269094
21: 0.138327
22: 0.447186
23: 0.411021
24: 0.000000
25: 0.037365
26: 0.000000
27: 0.031548
28: 0.012900
29: 0.067285
3: 0.146186
30: 0.004149
31: 0.000000
32: 0.190494
33: 0.210473
34: 0.003448
35: 0.014921
36: 0.007183
37: 0.013276
38: 0.018384
39: 0.032068
4: 0.396738
40: 0.033936
41: 0.050638
42: 0.008140
43: 0.008733
44: 0.000000
45: 0.056904
46: 0.026742
47: 0.004237
48: 0.021629
49: 0.043389
5: 0.307921
50: 0.022917
51: 0.005205
52: 0.007733
53: 0.145816
54: 0.057694
55: 0.010839
56: 0.033768
57: 0.134704
58: 0.019006
59: 0.125771
6: 0.385115
60: 0.045610
61: 0.301732
62: 0.245281
63: 0.172314
64: 0.098075
65: 0.004947
66: 0.098838
67: 0.015076
68: 0.208081
69: 0.056799
7: 0.075602
70: 0.000000
71: 0.076651
72: 0.121900
73: 0.007855
74: 0.299610
75: 0.035846
76: 0.000

In [ ]:
0.07290217209136615

In [84]:
import pandas as pd
coord=pred_final[:,:4].cpu().detach().numpy()
conf=pred_final[:,4:5].cpu().detach().numpy()
mat=np.hstack((conf,coord))

classes=pred_final[:,5:].max(1)[1].cpu().detach().numpy()
classes=np.array([classes]).T

mat=np.hstack((classes,mat))
mat=np.array(mat)

df=pd.DataFrame(mat,index=None,columns=None)
df[0]=df[0].apply(lambda x: int(x))

df.to_csv('test.txt',sep=' ',header=False,index=None)

AttributeError: 'tuple' object has no attribute 'cpu'

In [80]:
np.savetxt('test.txt',mat)

In [63]:
a=torch.tensor([1,2,2,3,4,2,3,1,4])
print(a.max(0))

torch.return_types.max(
values=tensor(4),
indices=tensor(8))


In [ ]:
import skimage.io as io

In [ ]:
image=(inp).squeeze(0)
image=np.array(image.cpu())
print(image.shape)
image =  image[:,:,::-1].transpose((1,2,0))
print(image.shape)

In [ ]:
import pandas as pd
import glob

all_files = glob.glob('plots/' + "/*.csv")

li = []

for filename in sorted(all_files):
    print(filename)
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=1, ignore_index=True)


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# gca stands for 'get current axis'

title_list=['AIoU_train','Loss_train','NClass_train','NConf_train','PClass','PConf']

fig = plt.figure(figsize=(30, 10))
fig.suptitle('KL for xy loss')
fig.subplots_adjust(hspace=0.3, wspace=-.6)
colors = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c']
i=2
k=0
while i <18:
    ax = fig.add_subplot(2, 9, i)
    frame.plot(x =1 , y = i,ax=ax,legend=False)
    ax.set_title(title_list[k])
    i=i+3
    k=k+1
plt.savefig('original.png')
plt.show()